# Label and Ordinal Encoding

## Introduction

Unlike One-Hot Encoding which creates multiple binary columns, **Label Encoding** and **Ordinal Encoding** convert categories to integer numbers.

### Label Encoding vs Ordinal Encoding

**Label Encoding:**
- Assigns integers to categories (0, 1, 2, ...)
- **No inherent order** assumed
- Simple mapping: Category → Number
- Used for target variables or tree-based models

**Ordinal Encoding:**
- Assigns integers based on **meaningful order**
- Preserves ranking relationship
- Used when categories have natural ordering
- Examples: Small < Medium < Large

### When to Use

| Scenario | Use Label | Use Ordinal |
|----------|-----------|-------------|
| **No order** (Color: Red, Blue) | ✅ With caution | ❌ |
| **Has order** (Size: S < M < L) | ❌ | ✅ |
| **Target variable** | ✅ | ✅ |
| **Tree models** | ✅ | ✅ |
| **Linear models** | ❌ | ⚠️ Only if truly ordinal |

### Advantages

✅ Simple and memory efficient
✅ No dimension increase (unlike OHE)
✅ Works well with tree-based models
✅ Preserves order for ordinal data
✅ Fast to compute

### Disadvantages

❌ Imposes false ordering (if data isn't ordinal)
❌ Not suitable for all algorithms (e.g., linear regression)
❌ Distance between numbers may be misinterpreted
❌ Can confuse models about relationships

Let's explore both encoding methods!

## Step 1: Import Libraries and Create Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import warnings
warnings.filterwarnings('ignore')

# Create dataset with both nominal and ordinal features
data = {
    'Customer_ID': range(1, 16),
    'Color': ['Red', 'Blue', 'Green', 'Red', 'Blue', 'Green', 'Red', 'Blue', 
              'Green', 'Red', 'Blue', 'Green', 'Red', 'Blue', 'Green'],
    'Size': ['Small', 'Medium', 'Large', 'Medium', 'Small', 'Large', 'Small',
             'Large', 'Medium', 'Small', 'Large', 'Medium', 'Large', 'Small', 'Medium'],
    'Education': ['High School', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'High School',
                  'Master', 'PhD', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'Master',
                  'High School', 'PhD'],
    'Rating': ['Poor', 'Good', 'Excellent', 'Good', 'Poor', 'Excellent', 'Good',
               'Excellent', 'Poor', 'Good', 'Excellent', 'Poor', 'Good', 'Excellent', 'Poor'],
    'Purchase': [100, 250, 400, 300, 150, 450, 200, 500, 180, 280, 520, 160, 320, 480, 140]
}

df = pd.DataFrame(data)

print("=" * 100)
print("DATASET WITH NOMINAL AND ORDINAL FEATURES")
print("=" * 100)
print(df)

print("\n" + "-" * 100)
print("FEATURE TYPES:")
print("-" * 100)
print("NOMINAL (No order):")
print("  • Color: Red, Blue, Green - no inherent ranking")
print("\nORDINAL (Has order):")
print("  • Size: Small < Medium < Large")
print("  • Education: High School < Bachelor < Master < PhD")
print("  • Rating: Poor < Good < Excellent")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Categorical Features Distribution', fontsize=16, fontweight='bold')

features = ['Color', 'Size', 'Education', 'Rating']
for idx, feature in enumerate(features):
    row = idx // 2
    col = idx % 2
    
    counts = df[feature].value_counts()
    axes[row, col].bar(counts.index, counts.values, color='skyblue', 
                       alpha=0.7, edgecolor='black', linewidth=2)
    axes[row, col].set_xlabel(feature, fontweight='bold', fontsize=12)
    axes[row, col].set_ylabel('Count', fontweight='bold', fontsize=12)
    
    feature_type = 'NOMINAL' if feature == 'Color' else 'ORDINAL'
    axes[row, col].set_title(f'{feature} Distribution ({feature_type})', 
                            fontweight='bold', fontsize=13)
    axes[row, col].tick_params(axis='x', rotation=45)
    axes[row, col].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()
print("\n" + "=" * 100)

## Method 1: Label Encoding

Label Encoding assigns a unique integer to each category automatically.

In [ ]:
# Label Encoding - Converts categories to integers automatically
from sklearn.preprocessing import LabelEncoder

# Apply Label Encoding to all categorical columns
le_color = LabelEncoder()
le_size = LabelEncoder()
le_education = LabelEncoder()
le_rating = LabelEncoder()

df['Color_LabelEncoded'] = le_color.fit_transform(df['Color'])
df['Size_LabelEncoded'] = le_size.fit_transform(df['Size'])
df['Education_LabelEncoded'] = le_education.fit_transform(df['Education'])
df['Rating_LabelEncoded'] = le_rating.fit_transform(df['Rating'])

print("=== Label Encoding Results ===\n")
print("Original Data vs Label Encoded Data:")
print(df[['Color', 'Color_LabelEncoded', 'Size', 'Size_LabelEncoded', 
          'Education', 'Education_LabelEncoded', 'Rating', 'Rating_LabelEncoded']].head(10))

print("\n" + "="*60)
print("Mapping for each feature:")
print("="*60)

# Show mapping for Color (Nominal)
print("\n1. Color Mapping:")
color_mapping = dict(zip(le_color.classes_, le_color.transform(le_color.classes_)))
for original, encoded in sorted(color_mapping.items()):
    print(f"   {original} → {encoded}")

# Show mapping for Size (Ordinal)
print("\n2. Size Mapping:")
size_mapping = dict(zip(le_size.classes_, le_size.transform(le_size.classes_)))
for original, encoded in sorted(size_mapping.items()):
    print(f"   {original} → {encoded}")
print("   ⚠️ Problem: Large(0) < Medium(1) < Small(2) - Incorrect order!")

# Show mapping for Education (Ordinal)
print("\n3. Education Mapping:")
education_mapping = dict(zip(le_education.classes_, le_education.transform(le_education.classes_)))
for original, encoded in sorted(education_mapping.items()):
    print(f"   {original} → {encoded}")
print("   ⚠️ Problem: Bachelor(0) < High School(1) < Master(2) < PhD(3) - Incorrect order!")

# Show mapping for Rating (Ordinal)
print("\n4. Rating Mapping:")
rating_mapping = dict(zip(le_rating.classes_, le_rating.transform(le_rating.classes_)))
for original, encoded in sorted(rating_mapping.items()):
    print(f"   {original} → {encoded}")
print("   ⚠️ Problem: Excellent(0) < Good(1) < Poor(2) - Incorrect order!")

# Visualize the encoding
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Label Encoding - Automatic Assignment (Alphabetical Order)', fontsize=16, fontweight='bold')

# Color
axes[0, 0].bar(df['Color'].unique(), [color_mapping[c] for c in df['Color'].unique()], color='skyblue')
axes[0, 0].set_xlabel('Original Categories')
axes[0, 0].set_ylabel('Encoded Values')
axes[0, 0].set_title('Color (Nominal) - OK for nominal data')
axes[0, 0].grid(axis='y', alpha=0.3)

# Size
size_order = ['Small', 'Medium', 'Large']
axes[0, 1].bar(size_order, [size_mapping[s] for s in size_order], color='salmon')
axes[0, 1].set_xlabel('Original Categories (Correct Order)')
axes[0, 1].set_ylabel('Encoded Values')
axes[0, 1].set_title('Size (Ordinal) - ❌ Wrong Order!')
axes[0, 1].grid(axis='y', alpha=0.3)

# Education
education_order = ['High School', 'Bachelor', 'Master', 'PhD']
axes[1, 0].bar(education_order, [education_mapping[e] for e in education_order], color='lightcoral')
axes[1, 0].set_xlabel('Original Categories (Correct Order)')
axes[1, 0].set_ylabel('Encoded Values')
axes[1, 0].set_title('Education (Ordinal) - ❌ Wrong Order!')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(axis='y', alpha=0.3)

# Rating
rating_order = ['Poor', 'Good', 'Excellent']
axes[1, 1].bar(rating_order, [rating_mapping[r] for r in rating_order], color='lightcoral')
axes[1, 1].set_xlabel('Original Categories (Correct Order)')
axes[1, 1].set_ylabel('Encoded Values')
axes[1, 1].set_title('Rating (Ordinal) - ❌ Wrong Order!')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("⚠️  KEY OBSERVATION:")
print("="*60)
print("Label Encoding assigns values in ALPHABETICAL ORDER, not based on")
print("the natural ordering of the categories. This is problematic for")
print("ordinal features where the order matters!")
print("\n✅ Use Label Encoding for: NOMINAL data (no inherent order)")
print("❌ Avoid for: ORDINAL data (use Ordinal Encoding instead)")

## Method 2: Ordinal Encoding

Ordinal Encoding allows you to specify the order of categories explicitly, which is essential for ordinal features.

In [ ]:
# Ordinal Encoding - Specify custom order for categories
from sklearn.preprocessing import OrdinalEncoder

# Define the correct order for each ordinal feature
size_order = [['Small', 'Medium', 'Large']]  # Small < Medium < Large
education_order = [['High School', 'Bachelor', 'Master', 'PhD']]  # Increasing level
rating_order = [['Poor', 'Good', 'Excellent']]  # Poor < Good < Excellent

# Create separate encoders for each ordinal feature
ordinal_encoder_size = OrdinalEncoder(categories=size_order)
ordinal_encoder_education = OrdinalEncoder(categories=education_order)
ordinal_encoder_rating = OrdinalEncoder(categories=rating_order)

# Apply Ordinal Encoding
df['Size_OrdinalEncoded'] = ordinal_encoder_size.fit_transform(df[['Size']])
df['Education_OrdinalEncoded'] = ordinal_encoder_education.fit_transform(df[['Education']])
df['Rating_OrdinalEncoded'] = ordinal_encoder_rating.fit_transform(df[['Rating']])

print("=== Ordinal Encoding Results ===\n")
print("Comparison: Label Encoding vs Ordinal Encoding")
print(df[['Size', 'Size_LabelEncoded', 'Size_OrdinalEncoded',
          'Education', 'Education_LabelEncoded', 'Education_OrdinalEncoded',
          'Rating', 'Rating_LabelEncoded', 'Rating_OrdinalEncoded']].head(10))

print("\n" + "="*60)
print("Correct Ordinal Encoding Mappings:")
print("="*60)

# Size mapping
print("\n1. Size Mapping:")
for i, category in enumerate(size_order[0]):
    print(f"   {category} → {i} ✅")
print("   Order: Small(0) < Medium(1) < Large(2) - Correct!")

# Education mapping
print("\n2. Education Mapping:")
for i, category in enumerate(education_order[0]):
    print(f"   {category} → {i} ✅")
print("   Order: High School(0) < Bachelor(1) < Master(2) < PhD(3) - Correct!")

# Rating mapping
print("\n3. Rating Mapping:")
for i, category in enumerate(rating_order[0]):
    print(f"   {category} → {i} ✅")
print("   Order: Poor(0) < Good(1) < Excellent(2) - Correct!")

# Visualize comparison between Label Encoding and Ordinal Encoding
fig, axes = plt.subplots(3, 2, figsize=(14, 12))
fig.suptitle('Label Encoding vs Ordinal Encoding Comparison', fontsize=16, fontweight='bold')

# Size comparison
size_categories = ['Small', 'Medium', 'Large']
axes[0, 0].bar(size_categories, [size_mapping[s] for s in size_categories], color='salmon', alpha=0.7)
axes[0, 0].set_xlabel('Size Categories')
axes[0, 0].set_ylabel('Encoded Values')
axes[0, 0].set_title('Size - Label Encoding (❌ Wrong Order)', fontweight='bold')
axes[0, 0].grid(axis='y', alpha=0.3)
axes[0, 0].set_ylim([0, 2.5])

axes[0, 1].bar(size_categories, range(len(size_categories)), color='lightgreen', alpha=0.7)
axes[0, 1].set_xlabel('Size Categories')
axes[0, 1].set_ylabel('Encoded Values')
axes[0, 1].set_title('Size - Ordinal Encoding (✅ Correct Order)', fontweight='bold')
axes[0, 1].grid(axis='y', alpha=0.3)
axes[0, 1].set_ylim([0, 2.5])

# Education comparison
education_categories = ['High School', 'Bachelor', 'Master', 'PhD']
axes[1, 0].bar(education_categories, [education_mapping[e] for e in education_categories], 
               color='salmon', alpha=0.7)
axes[1, 0].set_xlabel('Education Level')
axes[1, 0].set_ylabel('Encoded Values')
axes[1, 0].set_title('Education - Label Encoding (❌ Wrong Order)', fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(axis='y', alpha=0.3)
axes[1, 0].set_ylim([0, 3.5])

axes[1, 1].bar(education_categories, range(len(education_categories)), color='lightgreen', alpha=0.7)
axes[1, 1].set_xlabel('Education Level')
axes[1, 1].set_ylabel('Encoded Values')
axes[1, 1].set_title('Education - Ordinal Encoding (✅ Correct Order)', fontweight='bold')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(axis='y', alpha=0.3)
axes[1, 1].set_ylim([0, 3.5])

# Rating comparison
rating_categories = ['Poor', 'Good', 'Excellent']
axes[2, 0].bar(rating_categories, [rating_mapping[r] for r in rating_categories], 
               color='salmon', alpha=0.7)
axes[2, 0].set_xlabel('Rating')
axes[2, 0].set_ylabel('Encoded Values')
axes[2, 0].set_title('Rating - Label Encoding (❌ Wrong Order)', fontweight='bold')
axes[2, 0].grid(axis='y', alpha=0.3)
axes[2, 0].set_ylim([0, 2.5])

axes[2, 1].bar(rating_categories, range(len(rating_categories)), color='lightgreen', alpha=0.7)
axes[2, 1].set_xlabel('Rating')
axes[2, 1].set_ylabel('Encoded Values')
axes[2, 1].set_title('Rating - Ordinal Encoding (✅ Correct Order)', fontweight='bold')
axes[2, 1].grid(axis='y', alpha=0.3)
axes[2, 1].set_ylim([0, 2.5])

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("✅ KEY ADVANTAGE:")
print("="*60)
print("Ordinal Encoding respects the NATURAL ORDER of categories,")
print("which is crucial for ordinal features. This preserves the")
print("meaningful relationships between categories for ML models.")

## Impact on Machine Learning Models

Let's see how incorrect encoding affects model predictions.

In [ ]:
# Compare model performance with Label Encoding vs Ordinal Encoding
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Create a larger synthetic dataset for better comparison
np.random.seed(42)
n_samples = 200

# Generate data where Purchase depends on ordinal features
sizes = np.random.choice(['Small', 'Medium', 'Large'], n_samples, p=[0.3, 0.4, 0.3])
educations = np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], n_samples, p=[0.2, 0.4, 0.3, 0.1])
ratings = np.random.choice(['Poor', 'Good', 'Excellent'], n_samples, p=[0.2, 0.5, 0.3])
colors = np.random.choice(['Red', 'Blue', 'Green'], n_samples)

# Create Purchase based on ordinal values (higher education, larger size, better rating = higher purchase)
size_value = {'Small': 10, 'Medium': 20, 'Large': 30}
education_value = {'High School': 15, 'Bachelor': 30, 'Master': 45, 'PhD': 60}
rating_value = {'Poor': 5, 'Good': 15, 'Excellent': 25}

purchases = [size_value[s] + education_value[e] + rating_value[r] + np.random.normal(0, 10) 
             for s, e, r in zip(sizes, educations, ratings)]

# Create DataFrame
df_large = pd.DataFrame({
    'Size': sizes,
    'Education': educations,
    'Rating': ratings,
    'Color': colors,
    'Purchase': purchases
})

# Encode using both methods
# Label Encoding
le_size_large = LabelEncoder()
le_education_large = LabelEncoder()
le_rating_large = LabelEncoder()

df_large['Size_Label'] = le_size_large.fit_transform(df_large['Size'])
df_large['Education_Label'] = le_education_large.fit_transform(df_large['Education'])
df_large['Rating_Label'] = le_rating_large.fit_transform(df_large['Rating'])

# Ordinal Encoding
oe_size_large = OrdinalEncoder(categories=[['Small', 'Medium', 'Large']])
oe_education_large = OrdinalEncoder(categories=[['High School', 'Bachelor', 'Master', 'PhD']])
oe_rating_large = OrdinalEncoder(categories=[['Poor', 'Good', 'Excellent']])

df_large['Size_Ordinal'] = oe_size_large.fit_transform(df_large[['Size']])
df_large['Education_Ordinal'] = oe_education_large.fit_transform(df_large[['Education']])
df_large['Rating_Ordinal'] = oe_rating_large.fit_transform(df_large[['Rating']])

# Train models with both encoding methods
# Model 1: Using Label Encoding
X_label = df_large[['Size_Label', 'Education_Label', 'Rating_Label']]
y = df_large['Purchase']

X_train_label, X_test_label, y_train, y_test = train_test_split(X_label, y, test_size=0.3, random_state=42)

model_label = LinearRegression()
model_label.fit(X_train_label, y_train)
y_pred_label = model_label.predict(X_test_label)

# Model 2: Using Ordinal Encoding
X_ordinal = df_large[['Size_Ordinal', 'Education_Ordinal', 'Rating_Ordinal']]

X_train_ordinal, X_test_ordinal, _, _ = train_test_split(X_ordinal, y, test_size=0.3, random_state=42)

model_ordinal = LinearRegression()
model_ordinal.fit(X_train_ordinal, y_train)
y_pred_ordinal = model_ordinal.predict(X_test_ordinal)

# Compare performance
print("="*70)
print("MODEL PERFORMANCE COMPARISON")
print("="*70)

print("\n1. Model with LABEL ENCODING (Incorrect Order):")
print(f"   R² Score:  {r2_score(y_test, y_pred_label):.4f}")
print(f"   MAE:       ${mean_absolute_error(y_test, y_pred_label):.2f}")
print(f"   RMSE:      ${np.sqrt(mean_squared_error(y_test, y_pred_label)):.2f}")

print("\n2. Model with ORDINAL ENCODING (Correct Order):")
print(f"   R² Score:  {r2_score(y_test, y_pred_ordinal):.4f}")
print(f"   MAE:       ${mean_absolute_error(y_test, y_pred_ordinal):.2f}")
print(f"   RMSE:      ${np.sqrt(mean_squared_error(y_test, y_pred_ordinal)):.2f}")

improvement_r2 = ((r2_score(y_test, y_pred_ordinal) - r2_score(y_test, y_pred_label)) 
                  / r2_score(y_test, y_pred_label) * 100)
improvement_mae = ((mean_absolute_error(y_test, y_pred_label) - mean_absolute_error(y_test, y_pred_ordinal)) 
                   / mean_absolute_error(y_test, y_pred_label) * 100)

print(f"\n✅ Improvement with Ordinal Encoding:")
print(f"   R² Score improvement: {improvement_r2:.2f}%")
print(f"   MAE reduction: {improvement_mae:.2f}%")

# Visualize predictions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Label Encoding predictions
axes[0].scatter(y_test, y_pred_label, alpha=0.6, color='salmon')
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
axes[0].set_xlabel('Actual Purchase Amount')
axes[0].set_ylabel('Predicted Purchase Amount')
axes[0].set_title(f'Label Encoding\nR² = {r2_score(y_test, y_pred_label):.4f}', fontweight='bold')
axes[0].grid(alpha=0.3)

# Ordinal Encoding predictions
axes[1].scatter(y_test, y_pred_ordinal, alpha=0.6, color='lightgreen')
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
axes[1].set_xlabel('Actual Purchase Amount')
axes[1].set_ylabel('Predicted Purchase Amount')
axes[1].set_title(f'Ordinal Encoding\nR² = {r2_score(y_test, y_pred_ordinal):.4f}', fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("📊 INTERPRETATION:")
print("="*70)
print("The model with Ordinal Encoding performs better because it respects the")
print("natural ordering of categories. This allows the model to learn meaningful")
print("relationships: higher education → higher purchase, larger size → higher purchase.")
print("\nWith Label Encoding, the alphabetical order misleads the model, resulting")
print("in poorer predictions.")

## Decision Framework: Which Encoding to Use?

| Feature Type | Encoding Method | Reason | Example |
|-------------|----------------|--------|---------|
| **Nominal** (No order) | Label Encoding or One-Hot Encoding | Categories have no meaningful order | Color (Red, Blue, Green), Country, Gender |
| **Ordinal** (Has order) | Ordinal Encoding | Preserves natural ordering | Education (High School < Bachelor < Master), Size (S < M < L < XL), Rating (Poor < Good < Excellent) |
| **Binary** | Label Encoding | Only 2 categories, order doesn't matter | Yes/No, True/False, Male/Female |
| **High Cardinality Nominal** | Target Encoding, Frequency Encoding | Too many categories for OHE | City, Product ID, Customer ID |

### Best Practices

1. **Always identify feature type first**: Determine if your categorical feature is nominal or ordinal
2. **Use Ordinal Encoding for ordinal features**: Never use Label Encoding for ordinal data
3. **Specify correct order**: When using Ordinal Encoding, ensure the order reflects real-world meaning
4. **Consider tree-based models**: Decision trees can handle label encoded ordinal features better than linear models
5. **Document your encoding choices**: Keep track of which features were encoded and why

## Summary

### Key Takeaways

1. **Label Encoding**:
   - Assigns integers to categories automatically (alphabetical order)
   - ✅ Good for: Nominal features, binary features
   - ❌ Avoid for: Ordinal features with meaningful order
   - Simple but can mislead models on ordinal data

2. **Ordinal Encoding**:
   - Allows explicit specification of category order
   - ✅ Essential for: Ordinal features (Size, Education, Rating, etc.)
   - Preserves meaningful relationships between categories
   - Improves model performance on ordered data

3. **Critical Difference**:
   - Label: Alphabetical order (automatic, no control)
   - Ordinal: Custom order (manual, full control)

4. **Model Impact**:
   - Correct encoding → Better predictions, higher R² score
   - Incorrect encoding → Misleading relationships, poor performance

### Remember
**Always use Ordinal Encoding when your categorical feature has a natural order!**